# UNET Segmentation

<div style="font-size: 20px; line-height: 30px;">
Research Paper: <a href="https://arxiv.org/abs/1505.04597">U-Net: Convolutional Networks for Biomedical Image Segmentation</a>

<ul>
<li>UNet is a fully convolutional network(FCN) that does image segmentation. Its goal is to predict each pixel's class.</li>
<li>UNet is built upon the FCN and modified in a way that it yields better segmentation in medical imaging.</li>
</ul>
</div>

## Architecture

<img src="image/u-net-architecture.png">
<center> The block diagram of the U-Net architecture </center>

<div style="font-size: 20px; line-height: 30px;">
UNet Architecture has 3 parts:
<ol>
<li>Encoder </li>
<li>Decoder </li>
<li>Skip Connection</li>
</ol>
</div>

<img src="image/encoder-decoder.png">

## Simple UNET Architecture

<img src="image/unet-simple.png" align="left">

## Import

In [1]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

## Conv Block

In [2]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    return x

## Encoder Block

In [3]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

## Decoder Block

In [4]:
def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

## UNET

In [17]:
def build_unet(input_shape):
    inputs = Input(input_shape)
    
    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    
    """ Bridge """
    b1 = conv_block(p4, 1024)
    
    """ Decoder """
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    
    model = Model(inputs, outputs, name="UNET")
    return model

## Run the Model

In [18]:
input_shape = (256, 256, 3)
model = build_unet(input_shape)

In [19]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_79 (Conv2D)              (None, 256, 256, 64) 1792        input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_78 (BatchNo (None, 256, 256, 64) 256         conv2d_79[0][0]                  
__________________________________________________________________________________________________
activation_78 (Activation)      (None, 256, 256, 64) 0           batch_normalization_78[0][0]     
_______________________________________________________________________________________________